# Agents with Tool Calling

In [59]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv() # Load environment variables from .env file

az_o3_model_client = AzureOpenAIChatCompletionClient(
    model = "o3-mini",
    azure_endpoint = os.getenv("AZURE_OPENAI_API_URL"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    azure_deployment = "o3-mini",
    api_version = "2024-12-01-preview"
)

## Build in tools
### HttpTool

Using HttpTool to get an WorkItem's detail

#### Configuration tool

In [60]:
from autogen_ext.tools.http import HttpTool
from urllib.parse import quote
work_item_schema = {
    "type": "object",
    "properties": {
        "org": {
            "type": "string",
            "description": "The Azure DevOps organization name."
        },
        "project": {
            "type": "string",
            "description": "The Azure DevOps project name."
        },
        "id": {
            "type": "string",
            "description": "The work item ID."
        }
    },
    "required": ["org", "project", "id"],
}
token = os.getenv("ADO_PERSONAL_ACCESS_TOKEN")

get_work_item_tool = HttpTool(
    name="get_work_item",
    description="Get work item details.",
    method="GET",
    scheme="https",
    host="dev.azure.com",
    path="/{org}/{project}/_apis/wit/workitems/{id}",
    port=443,
    headers={
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    },
    json_schema=work_item_schema,
)

Create Agent with tool

In [63]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_agentchat.ui import Console

agent = AssistantAgent(
    name = "assistant",
    system_message = "You are an experienced assistant with deep knowledge of Azure DevOps (ADO), particularly skilled in managing Work Items efficiently and accurately.",
    model_client = az_o3_model_client,
    tools = [get_work_item_tool],
)
async def request_work_item_details():
    # Example request to get work item details
    response = await agent.on_messages(
            [TextMessage(content="Get the details of work item '4850952' in project 'OneCRM' in organization 'dynamicscrm'", source="user")],
            CancellationToken(),
        )
    print(response.chat_message.content)

await request_work_item_details()

{"id":4850952,"rev":8,"fields":{"System.AreaPath":"OneCRM\\CXA Portfolio\\Eng-CES-Voice\\Voice Eng Enablement\\Voice-Pioneers","System.TeamProject":"OneCRM","System.IterationPath":"OneCRM\\Bromine\\CY25Q2\\2Wk\\2Wk01 (Apr 06 - Apr 19)","System.WorkItemType":"User Story","System.State":"Active","System.Reason":"Implementation started","System.AssignedTo":{"displayName":"Wei Jen Lu","url":"https://spsprodwus21.vssps.visualstudio.com/A39971328-31a2-4457-896a-6cbe4d8b0aca/_apis/Identities/8c4edfb4-9f82-6960-bdf1-e054109ba4ba","_links":{"avatar":{"href":"https://dev.azure.com/dynamicscrm/_apis/GraphProfile/MemberAvatars/aad.OGM0ZWRmYjQtOWY4Mi03OTYwLWJkZjEtZTA1NDEwOWJhNGJh"}},"id":"8c4edfb4-9f82-6960-bdf1-e054109ba4ba","uniqueName":"weijenlu@microsoft.com","imageUrl":"https://dev.azure.com/dynamicscrm/_apis/GraphProfile/MemberAvatars/aad.OGM0ZWRmYjQtOWY4Mi03OTYwLWJkZjEtZTA1NDEwOWJhNGJh","descriptor":"aad.OGM0ZWRmYjQtOWY4Mi03OTYwLWJkZjEtZTA1NDEwOWJhNGJh"},"System.CreatedDate":"2025-03-27T18:0